In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/Group_A_Project
%ls

/content/drive/MyDrive/Colab Notebooks/Group_A_Project
 Amazon_Unlocked_Mobile.csv                   'Milestone (Project 開發流程).gdoc'
 Amazon_Unlocked_Mobile.csv_bert.pth           monthly_data.csv
 APIR_commemt_data.csv                         my_database/
 APIR_commemt_data.gsheet                      Poster.pptx
 BERT_Visualization.ipynb                     'Poster Slide.gslides'
'ChatGPT&APIR_commemt_respond_clean.csv'       poster_template_2024.pptx
'ChatGPT&APIR_commemt_respond.csv'             predictions.csv
'ChatGPT&MACRUMORS_commemt_repond_clean.csv'  'Pre-poster Slide.gslides'
'ChatGPT og data'/                             Project.ipynb
'ChatGPT&TrustPilot_commemt_out.csv'          'Project Timeline.gsheet'
'Copy of Amazon_Unlocked_Mobile.csv'           RAG.ipynb
'Copy of RAG.ipynb'                            results.csv
'Copy of RAG test.ipynb'                       review_responses.csv
 IMDB.ipynb                                    stopwords.txt
 Lora-Medium-5.ttf          

In [3]:
%pip install langchain-community
%pip install langchain_openai
%pip install chromadb
!pip install torch torchvision torchaudio
!pip install sentence-transformers

In [4]:
import pandas as pd
from langchain.schema import Document
from langchain.document_loaders import TextLoader
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
documents = df['Reviews'].dropna().tolist()
chunks = [Document(page_content=line) for line in documents]

In [5]:
import os
API_KEY = ''
os.environ['OPENAI_API_KEY'] = API_KEY

In [6]:
import torch
print("GPU available:", torch.cuda.is_available())


GPU available: True


In [8]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [9]:
CHROMA_PATH = 'my_database'

# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')  # upload GPU
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # upload GPU

# Function to embed documents
def embed_documents(documents):
    texts = [doc.page_content for doc in documents]  # extract
    embeddings = embedding_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
    return embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:

# Embed chunks
embeddings = embed_documents(chunks)



Batches:   0%|          | 0/12931 [00:00<?, ?it/s]

In [11]:
# MAC Rumors
# Read reviews from CSV
csv_path = 'MacRumors - MacRumors_test.csv'
df = pd.read_csv(csv_path)
reviews_list = df['content'].dropna().tolist()

# # Trustpilot
# # Read reviews from CSV
# csv_path = 'Trustpilot_commemt_data.csv'
# df = pd.read_csv(csv_path)
# reviews_list = df['content'].dropna().tolist()

In [12]:
# # AppleInsider
# # Read reviews from CSV
# csv_path = 'APIR_commemt_data.csv'
# df = pd.read_csv(csv_path)
# reviews_list = df['content'].dropna().tolist()

In [13]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:


{review}

---

Answer the question based on the above context: {question}
"""

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

model = ChatOpenAI()


# Function to process reviews in batch
def process_reviews_in_batch(reviews_batch):
    batch_prompt = "\n\n".join([f"Review: {review}\nIs this line of review positive or negative? (Consider every line seperately and answer only 'positive', 'negative' or 'neutral')" for
                                review in reviews_batch])
    response = model.predict(batch_prompt)
    responses = response.split('\n')
    if len(responses) < len(reviews_batch):
        responses.extend(['neutral'] * (len(reviews_batch) - len(responses)))

    return responses

# Function to process reviews in batch
# def process_reviews_in_batch(reviews_batch):
#     batch_prompt = "\n\n".join([f"Review: {review}\nIs this review positive or negative?(Answer only 'positive' or 'negative' or 'neutral' or 'I am not really sure')" for review in reviews_batch])
#     # batch_prompt = "\n\n".join([f"Review: {review}\nIs this review positive or negative?(Answer only 'positive' or 'negative' or 'neutral' or 'I am not really sure')" for review in reviews_batch])

#     response = model.predict(batch_prompt)
#     return response

batch_size = 5
batched_reviews = [reviews_list[i:i + batch_size] for i in range(0, len(reviews_list), batch_size)]


In [15]:
# Process reviews in batches
results = []
for batch in batched_reviews:
    responses = process_reviews_in_batch(batch)
    # results.append(response)
    if len(responses) == 0 or len(responses) > 15  :
        response = 'Unable to determine sentiment'

    for review, response in zip(batch, responses):
        results.append({'review': review, 'response': response})

# Output results
for i, result in enumerate(results):
    print(f"Review {i + 1}: {result['review']} \nResponse: {result['response']}\n")
    # print(f"Review{i + 1}{reviews_list[i:i + batch_size]} \nResponse:{result}\n")
    # review label time

<ipython-input-14-406eee2e9e11>:11: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model.predict(batch_prompt)


Streaming output truncated to the last 5000 lines.
Response: Review 1: Neutral

Review 44552: So youre keeping both? 
Response: Review 2: Neutral

Review 44553: Yep, thats the plan but i just sold the S7 SSG. 
Response: Review 3: Neutral

Review 44554: Nice combo Haven't tried Sunday Surprise - does it mean it changes only on Sundays? 
Response: Review 4: Neutral

Review 44555: Why? 
Response: Review 5: Neutral

Review 44556: And you know this because? 
Response: 1. Neutral

Review 44557: Because of a nice fellow Austrian dude who made me aware of that fact thanks my friend 
Response: 2. Neutral

Review 44558: Because I only wore it when the Ultra was charging and I kept the S5 Ceramic when I need it dressyProbably going to wear the Ultras on a day by day alternating basis. 
Response: 3. Neutral

Review 44559: Lol rotating 2 ultras. Congrats on the Ultra 2! 
Response: 4. Positive

Review 44560: Awesome. I'll probably never own an Ultra but I learned something today.I suppose it's diffi

In [16]:
df_results = pd.DataFrame(results)

# 保存为 CSV 文件
csv_filename = 'ChatGPT&MACRUMORS_commemt_repond.csv'
# csv_filename = 'ChatGPT&TrustPilot_commemt_repond.csv'
# csv_filename = "ChatGPT&APIR_commemt_respond.csv"
df_results.to_csv(csv_filename, index=False)

print(f"Results have been saved to {csv_filename}")

Results have been saved to ChatGPT&MACRUMORS_commemt_repond.csv


Above will save in drive

In [ ]:
# Output results
for i, result in enumerate(results):
    print(f"Review {i + 1}: {result['review']} \nResponse: {result['response']}\n")
    # print(f"Review{i + 1}{reviews_list[i:i + batch_size]} \nResponse:{result}\n")
    # review label time

In [ ]:
df_results = pd.DataFrame(results)

# 保存为 CSV 文件
csv_filename = 'ChatGPT&MACRUMORS_commemt_repond.csv'
# csv_filename = "ChatGPT&APIR_commemt_respond.csv"
df_results.to_csv(csv_filename, index=False)

print(f"Results have been saved to {csv_filename}")

In [ ]:

csv_file = "ChatGPT&APIR_commemt_respond.csv"
with open('ChatGPT&APIR_commemt_respond.csv', 'a', newline='', encoding='utf-8') as file:
    fieldnames = [ 'review', 'response']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    results.append({ "review": review,"response":response})
    writer.writerow({'review': review,'response':response})

csv_file = "ChatGPT&APIR_commemt_respond.csv"
# /content/drive/My Drive/Colab\ Notebooks/Group_A_Project/
csv_file = "ChatGPT&APIR_commemt_respond.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Review", "response"])
    writer.writeheader()
    for data in results:
        writer.writerow(data)

print(f"Data has been saved to {csv_file}")

In [ ]:
import google.colab.files as files
files.download('ChatGPT&TrustPilot_commemt_repond.csv')